# G2Audit Guide - Summary Data

The `getSummaryData()` and `getSummaryDataDirect()` methods obtain general summary data about your current audit session.

More information:

1. [GitHub repository](https://github.com/Senzing/docker-jupyter)

## Prepare environment

In [ ]:
import os
import sys
import json

# For RenderJSON

import uuid
from IPython.display import display_javascript, display_html, display

### Helper class for JSON Rendering

In [ ]:
class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        elif isinstance(json_data, bytearray):
            self.json_str = json_data.decode()
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height:100%; width:100%; background-color: LightCyan"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### System path

Update system path.

In [ ]:
python_path = "{0}/python".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))
sys.path.append(python_path)

### Initialize variables

Create variables to be used as `getSummaryData()` parameters.

In [ ]:
module_name = 'pyG2AuditForGetSummary'

config_path = os.environ.get("SENZING_ETC_DIR", "/etc/opt/senzing")
support_path = os.environ.get("SENZING_DATA_VERSION_DIR", "/opt/senzing/data")

resource_path = "{0}/resources".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))

sql_connection = os.environ.get(
    "SENZING_SQL_CONNECTION", "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db")

verbose_logging = False

senzing_config_dictionary = {
    "PIPELINE": {
        "CONFIGPATH": config_path,        
        "SUPPORTPATH": support_path,
        "RESOURCEPATH": resource_path
    },
    "SQL": {
        "CONNECTION": sql_connection,
    }
}

senzing_config_json = json.dumps(senzing_config_dictionary)

## G2Audit

The G2Audit API allows you to pull data reporting data back on how one data source is related to another.
Once you are done loading one or more data sources,
use this API to query information that can be useful for reporting, browsing, or auditing results.

In [ ]:
from G2Audit import G2Audit

### Initialization

To start using Senzing G2Audit, create and initialize an instance.
This should be done once per process.
The `initV2()` method accepts the following parameters:

- **module_name:** A short name given to this instance of the G2Audit object.
- **config_json:** A JSON string containing configuration parameters.
- **verbose_logging:** A boolean which enables diagnostic logging.

Calling this function will return "0" upon success.

In [ ]:
g2_audit = G2Audit()
return_code = g2_audit.initV2(module_name, senzing_config_json, verbose_logging)

print("Return Code: {0}".format(return_code))

### openSession()

Create a new session.

In [ ]:
g2_audit_session = g2_audit.openSession()

### getSummaryData()

Call G2Audit's `getSummaryData()` method and pretty-print results.

The `getSummaryData()` method accepts the following parameters:

- **g2_audit_session:** A long integer representing a previously-created G2Audit object.
- **response_list:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
response_bytearray = bytearray()

return_code = g2_audit.getSummaryData(g2_audit_session, response_bytearray)

print("Return Code: {0}".format(return_code))
RenderJSON(response_bytearray)

### closeSession()

In [ ]:
g2_audit.closeSession(g2_audit_session)

### getSummaryDataDirect()

Call G2Audit's `getSummaryDataDirect()` method and pretty-print results.

The `getSummaryDataDirect()` method accepts the following parameters:

- **response_list:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
response_bytearray = bytearray()

return_code = g2_audit.getSummaryDataDirect(response_bytearray)

print("Return Code: {0}".format(return_code))
RenderJSON(response_bytearray)